# Machine Learning

Credit scoring in insurance companies
- huge data set
- features are: age, income, default

Train machine learning algorithm on the data set and  learns the relationship between the features and prediects

Predicting Stock Prices
Today   Yesterday  Day Before 
Lag #1 | Lag #2 | Lag # 3 | Direction
    r11|     r12|      r13| UP
    r21|     r22|      r23| DOWN
#### These are the Features ^^
 
## Logistic Regression

We have the features, the lagged percentage returns of a given stock 
we have a categorical dependent variable 
logistic regression provides a probability

p(y=UP | L1L2L3 ) = eB0 + l1b1 + l2b2 + l3b3 / 1+ e( eB0 + l1b1 + l2b2 + l3b3 )

linear regression is not workign this time: we are after probabilities between [0,1] for all values of L

with linear regression we would obtain negative values

What are the B values
- this is what we are after fitting the model means finding the B coefficients
- there are several methods to find these values: with gradient descending 
    


In [3]:
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LogisticRegression
import pandas_datareader.data as web
from sklearn.metrics import confusion_matrix

In [22]:
stock_symbol = 'JPM'
start_date = '01/01/2010'
end_date = '12/31/2019'
lags = 4
df = web.DataReader(stock_symbol, "yahoo", start_date, end_date)
df.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2010-01-04,42.990002,41.669998,41.790001,42.849998,35460500.0,32.941040
2010-01-05,43.840000,42.779999,42.790001,43.680000,41208300.0,33.579121
2010-01-06,44.090000,43.310001,43.450001,43.919998,27729000.0,33.763607
2010-01-07,45.119999,43.610001,43.790001,44.790001,44864700.0,34.432430
2010-01-08,44.700001,44.080002,44.369999,44.680000,33110100.0,34.347870


In [23]:
dflag = pd.DataFrame(index=df.index)
dflag["Today"] = df["Adj Close"]
dflag["Volume"] = df["Volume"]
dflag.head()

,Today,Volume
Date,,
2010-01-04,32.941040,35460500.0
2010-01-05,33.579121,41208300.0
2010-01-06,33.763607,27729000.0
2010-01-07,34.432430,44864700.0
2010-01-08,34.347870,33110100.0


In [24]:
# create the shifted l value series of prior trading periods adj close values
for i in range(0,lags):
    dflag["Lag%s" % str(i+1)] = df["Adj Close"].shift(i+1)

In [25]:
# create returns 
dfret = pd.DataFrame(index=dflag.index)
dfret["Volume"] = dflag["Volume"]
dfret["Today"] = dflag["Today"].pct_change()*100
dfret.head()


,Volume,Today
Date,,
2010-01-04,35460500.0,NaN
2010-01-05,41208300.0,1.937038
2010-01-06,27729000.0,0.549408
2010-01-07,44864700.0,1.980900
2010-01-08,33110100.0,-0.245584


In [26]:
for i in range(0,lags):
    dfret["Lags%s"%str(i+1)] = dflag["Lag%s"%str(i+1)].pct_change()*100

# Direction column
dfret["Direction"] = np.sign(dfret["Today"])

dfret.head()

C:\Users\Goliath\miniconda3\envs\quantitive_finance\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: invalid value encountered in sign
  result = getattr(ufunc, method)(*inputs, **kwargs)


,Volume,Today,Lags1,Lags2,Lags3,Lags4,Direction
Date,,,,,,,
2010-01-04,35460500.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,41208300.0,1.937038,NaN,NaN,NaN,NaN,1.0
2010-01-06,27729000.0,0.549408,1.937038,NaN,NaN,NaN,1.0
2010-01-07,44864700.0,1.980900,0.549408,1.937038,NaN,NaN,1.0
2010-01-08,33110100.0,-0.245584,1.980900,0.549408,1.937038,NaN,-1.0


In [27]:
dfret.drop(dfret.index[:5], inplace=True)

In [28]:
# Build Train and Test sets
X = dfret[["Lags1","Lags2","Lags3","Lags4"]]
y = dfret["Direction"]

start_test = datetime(2017,1,1)
X_train = X[X.index < start_test]
X_test = X[X.index >= start_test]
y_train = y[y.index < start_test]
y_test = y[y.index >= start_test]

# we use Logistic Regression as the model
model = LogisticRegression()

# fit the model
model.fit(X_train, y_train)

#make ana rray of predictions on the test set
predictions = model.predict(X_test)

# Score the model
model.score(X_test, y_test)

0.4973474801061008

In [29]:
# Confustion Matrix 
#Diagonal elemetns the correct classifications
#Off-Diagonal incorrect predictions
confusion_matrix(predictions, y_test)

array([[ 70,   0,  83],
       [  0,   0,   0],
       [292,   4, 305]], dtype=int64)

In [30]:
# Quantitative Analysis has an assumption all financial quantities such as stock prices or interest rates has random behavior. 
# We have to use randomness in our models - stochastic calculus and stochastic differential equations are needed.